In [1]:
import torch
from sklearn import metrics
from tabulate import tabulate
from torch.utils.data import DataLoader
from tqdm import tqdm

from model import TrackModel
from utils.dataset import JSONTrackDataset

In [2]:
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')
print('Using device:', device)
print()

# Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0) / 1024 ** 3, 1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0) / 1024 ** 3, 1), 'GB')

Using device: cpu



In [3]:
train_dataset = JSONTrackDataset('data/train.json')
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=50, num_workers=7, pin_memory=True,
                              drop_last=True)

val_dataset = JSONTrackDataset('data/val.json')
val_dataloader = DataLoader(val_dataset, shuffle=True, batch_size=50, num_workers=7, pin_memory=True, drop_last=True)

test_dataset = JSONTrackDataset('data/test.json')
test_dataloader = DataLoader(test_dataset, shuffle=True, batch_size=50, num_workers=7, pin_memory=True, drop_last=True)

In [4]:
model = TrackModel(5, 4).to(device)
EPOCHS_COUNT = 20

optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)
criterion = torch.nn.MSELoss(reduction='mean').to(device)

In [5]:
for epoch in range(EPOCHS_COUNT):
    # training
    loss_sum = 0
    ep_preds = []
    ep_labels = []
    for data_row in tqdm(train_dataloader, desc='Epoch {}/{}'.format(epoch, EPOCHS_COUNT - 1), ncols=80):
        data = data_row['data'].to(device)
        label = data_row['label'].to(device)

        optimizer.zero_grad()

        y_pred = model(data)
        loss = criterion(y_pred, label)
        loss_sum += loss.item()

        np_label = label.argmax(dim=1).data.numpy()
        np_preds = y_pred.argmax(dim=1).data.numpy()

        for i in np_label:
            ep_labels.append(i)
        for i in np_preds:
            ep_preds.append(i)

        loss.backward()
        optimizer.step()

    accuracy = metrics.accuracy_score(ep_labels, ep_preds)
    recall = metrics.recall_score(ep_labels, ep_preds, average='micro')
    precision = metrics.precision_score(ep_labels, ep_preds, average='micro')
    f1 = metrics.f1_score(ep_labels, ep_preds, average='micro')
    roc_auc = metrics.roc_auc_score(ep_labels, ep_preds, average='micro')

    print('')
    print(tabulate([
        ['Train', loss_sum / len(train_dataloader), accuracy, recall, precision, f1, roc_auc]
    ], headers=['Stage', 'Loss', 'Accuracy', 'Recall', 'Precision', 'F1', 'ROC-AUC'], tablefmt='orgtbl'))
    print('')

    # validation
    loss_sum = 0
    ep_preds = []
    ep_labels = []
    for val_row in tqdm(val_dataloader, desc='Val', ncols=80):
        data = val_row['data'].to(device)
        label = val_row['label'].to(device)

        y_pred = model(data)

        np_label = label.argmax(dim=1).data.numpy()
        np_preds = y_pred.argmax(dim=1).data.numpy()

        for i in np_label:
            ep_labels.append(i)
        for i in np_preds:
            ep_preds.append(i)

        loss = criterion(y_pred, label)
        loss_sum += loss.item()

    accuracy = metrics.accuracy_score(ep_labels, ep_preds)
    recall = metrics.recall_score(ep_labels, ep_preds, average='micro')
    precision = metrics.precision_score(ep_labels, ep_preds, average='micro')
    f1 = metrics.f1_score(ep_labels, ep_preds, average='micro')
    roc_auc = metrics.roc_auc_score(ep_labels, ep_preds, average='micro')

    print('')
    print(tabulate([
        ['Validation', loss_sum / len(train_dataloader), accuracy, recall, precision, f1, roc_auc]
    ], headers=['Stage', 'Loss', 'Accuracy', 'Recall', 'Precision', 'F1', 'ROC-AUC'], tablefmt='orgtbl'))
    print('')

Val:   0%|                                              | 0/375 [00:00<?, ?it/s]


| Stage   |   Loss |   Accuracy |    Recall |   Precision |        F1 |   ROC-AUC |
|---------+--------+------------+-----------+-------------+-----------+-----------|
| Train   | 960.28 |  0.0225933 | 0.0225933 |   0.0225933 | 0.0225933 |  0.499913 |



Epoch 1/19:   0%|                                      | 0/1126 [00:00<?, ?it/s]


| Stage      |     Loss |   Accuracy |   Recall |   Precision |      F1 |   ROC-AUC |
|------------+----------+------------+----------+-------------+---------+-----------|
| Validation | 0.170668 |    0.02192 |  0.02192 |     0.02192 | 0.02192 |       0.5 |



Val:   0%|                                              | 0/375 [00:00<?, ?it/s]

...


| Stage      |       Loss |   Accuracy |   Recall |   Precision |      F1 |   ROC-AUC |
|------------+------------+------------+----------+-------------+---------+-----------|
| Validation | 0.00994577 |    0.97808 |  0.97808 |     0.97808 | 0.97808 |       0.5 |



Val:   0%|                                              | 0/375 [00:00<?, ?it/s]


| Stage   |      Loss |   Accuracy |   Recall |   Precision |       F1 |   ROC-AUC |
|---------+-----------+------------+----------+-------------+----------+-----------|
| Train   | 0.0283681 |   0.978828 | 0.978828 |    0.978828 | 0.978828 |       0.5 |



Val: 100%|███████████████████████████████████| 375/375 [00:00<00:00, 554.38it/s]



| Stage      |       Loss |   Accuracy |   Recall |   Precision |      F1 |   ROC-AUC |
|------------+------------+------------+----------+-------------+---------+-----------|
| Validation | 0.00937654 |    0.97808 |  0.97808 |     0.97808 | 0.97808 |       0.5 |



In [6]:
loss_sum = 0
ep_preds = []
ep_labels = []
for data_row in tqdm(test_dataloader, desc='Test', ncols=80):
    data = data_row['data'].to(device)
    label = data_row['label'].to(device)

    optimizer.zero_grad()

    y_pred = model(data)
    loss = criterion(y_pred, label)
    loss_sum += loss.item()

    np_label = label.argmax(dim=1).data.numpy()
    np_preds = y_pred.argmax(dim=1).data.numpy()

    for i in np_label:
        ep_labels.append(i)
    for i in np_preds:
        ep_preds.append(i)

accuracy = metrics.accuracy_score(ep_labels, ep_preds)
recall = metrics.recall_score(ep_labels, ep_preds, average='micro')
precision = metrics.precision_score(ep_labels, ep_preds, average='micro')
f1 = metrics.f1_score(ep_labels, ep_preds, average='micro')
roc_auc = metrics.roc_auc_score(ep_labels, ep_preds, average='micro')

print('')
print(tabulate([
    ['Test', loss_sum / len(train_dataloader), accuracy, recall, precision, f1, roc_auc]
], headers=['Stage', 'Loss', 'Accuracy', 'Recall', 'Precision', 'F1', 'ROC-AUC'], tablefmt='orgtbl'))
print('')

Test: 100%|██████████████████████████████████| 375/375 [00:00<00:00, 660.99it/s]



| Stage   |       Loss |   Accuracy |   Recall |   Precision |       F1 |   ROC-AUC |
|---------+------------+------------+----------+-------------+----------+-----------|
| Test    | 0.00895241 |   0.979627 | 0.979627 |    0.979627 | 0.979627 |       0.5 |

